# <center>Big Data &ndash; Exercises &ndash; Solution</center>
## <center>Fall 2023 &ndash; Week 3 &ndash; ETH Zurich</center>

## Introduction
This week we will cover mostly theoretical aspects of Hadoop and HDFS and we will discuss advantages and limitations of different storage models.

#### What is Hadoop?
Hadoop provides a **distributed file system** and a
**framework for the analysis and transformation** of very **large**
data sets using the MapReduce paradigm.

Several components are part of this framework. In this course you will study HDFS, MapReduce and HBase while this exercise focuses on HDFS and storage models.


| *Component*                |*Description*  |*First developer*  |
|----------------------------------------------|---|---|
| **HDFS**                  |Distributed file system  |Yahoo!  |
| **MapReduce**   |Distributed computation framework   |Yahoo!  |
| **HBase**           | Column-oriented table service  |Powerset (Microsoft)  |
| Pig  | Dataflow language and parallel execution framework  |Yahoo!   |
| Hive            |Data warehouse infrastructure   |Facebook  |
| ZooKeeper    |Distributed coordination service   |Yahoo!  |
| Chukwa  |System for collecting management data   |Yahoo!  |
| Avro                |Data serialization system   |Yahoo! + Cloudera  |

## 1. The Hadoop Distributed File System
### 1.1 &ndash; State which of the following statements are true:

1. The HDFS namespace is a hierarchy of files and directories.

1. In HDFS, each block of the file is either 64 or 128 megabytes depending on the version and distribution of Hadoop in use, and this *cannot* be changed.

1. A client wanting to write a file into HDFS, first contacts the NameNode, then sends the data to it. The NameNode will write the data into multiple DataNodes in a pipelined fashion. 

1. A DataNode may execute multiple application tasks for different clients concurrently.

1. The cluster can have thousands of DataNodes and tens of thousands of HDFS clients per cluster.

1. HDFS NameNodes keep the namespace in DRAM and on disk.

1. The locations of block replicas are part of the persistent checkpoint that the NameNode stores in its local file system.

1. If the block size is set to 64 megabytes, storing a file of 80 megabytes will actually require 128 megabytes of physical memory (2 blocks of 64 megabytes each). 


**Solution**

1. True, in contrast with the Object Storage logic model, HDFS is designed to handle a relatively small amount of huge files. A hierarchical file system can therefore be handled efficiently by a single NameNode.

1. False, the default size is either 64 or 128 megabytes but this can be easily changed in the configuration.

1. False, the client writes data to the DataNodes. No data goes through the NameNode.

1. True, each DataNode may execute multiple application tasks concurrently.

1. True, since each DataNode can execute multiple tasks concurrently, there may be more clients than DataNodes.

1. True, and an image of such namespace is also persisted on disk.

1. False, the locations of block replicas may change over time and are not part of the persistent checkpoint.

1. False, the size of the data file equals the actual length of the block and does not require extra space to round it up to the nominal block size as in traditional file systems. Therefore 80 megabytes will be stored as a block of 64 megabytes + a block of 16 megabytes.

### 1.2 &ndash; A typical filesystem block size is 4096 bytes. How large is a block in HDFS? List at least two advantages of such choice.

**Solution**

Typical size for a block is either 64 or 128 megabytes. A large block size offers several important advantages. 

1. It minimizes the cost of seeks. If the block is large enough, the time it takes to transfer the data from the disk can be significantly longer than the time to seek to the start of the block. Thus, transferring a large file made of multiple blocks operates at the disk transfer rate.

2. It reduces clients' need to interact with the NameNode because reads and writes on the same block require only one initial request to the NameNode for location information. The reduction is significant for workloads where applications mostly read and write large files sequentially. 

3. Since on a large block, a client is more likely to perform many operations on the given block, it can reduce network overhead by keeping a persistent TCP connection to the DataNode over an extended period of time. 

4. It reduces the size of the metadata stored on the NameNode. This allows us to keep the metadata in memory.

### 1.3 &ndash; How does the hardware cost grow as function of the amount of data we need to store in a Distributed File System such as HDFS? Why?


**Solution**

**Linearly**. HDFS is designed taking machine failure into account, and therefore DataNodes do not need to be (highly expensive) highly reliable machines. Instead standard commodity hardware can be used. Moreover the number of nodes can be increased as soon as it becomes necessary, avoiding wasting of resources when the amount of data is still limited. This is indeed the main advantage of scaling out compared to scaling up, which has exponential cost growth.


### 1.4 &ndash; Single Point of Failure

1. Which component is the main single point of failure in Hadoop?

1. What is the Secondary NameNode?

**Solution**


1. Prior to Hadoop 2.0.0, the **NameNode was a single point of failure**. While the loss of any other machine (intermittently or permanently) does not result in data loss, NameNode loss results in cluster unavailability. The permanent loss of NameNode data would render the cluster's HDFS inoperable.
The HDFS High Availability feature addresses the above problems by providing the option of running two redundant NameNodes in the same cluster in an Active/Passive configuration with a hot standby. 

1. The Secondary NameNode is a node that merges the fsimage and the edits log files periodically and keeps edits log size within a limit. This allows the NameNode to start up faster in case of failure, but the Secondary NameNode is not a redundant NameNode. Over the years, the HDFS team kept improving on the "alternative" name nodes and came up almost every year with a new name with new functionality improving on the former ones. In the lecture, we discuss the latest up-to-date variant, standby namenodes, saying all the others (secondary, checkpoint, backup...) are just "HDFS archeology". For the possible configurations of namenodes, see [the official HDFS user guide](https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-hdfs/HdfsUserGuide.html#Secondary_NameNode). 

### 1.5 &ndash; Scalability, Durability and Performance on HDFS
Explain how HDFS accomplishes the following requirements:

1. Scalability

1. Durability

1. High sequential read/write performance

**Solution**

1. Scalability: by partitioning files into blocks and distributing them to many servers operating in parallel, HDFS can scale to potentially a large number of files of any size. By adding more DataNodes the storage capacity of the system can be increased arbitrarily. It has been demonstrated to scale beyond tens of petabytes (PB). More importantly, it does so with linear performance characteristics and cost.

1. Durability: HDFS creates multiple copies of each block (by default 3, on different racks) to minimize the probability of data loss.

1. High sequential read/write performance: by splitting huge files into blocks and spreading these into multiple machines. This makes parallel reads possible (accessing different nodes at the same time) either by using multiple clients or by using a distributed data processing framework such as MapReduce.

## 2. File I/O operations and replica management.


### 2.1 &ndash; Replication policy
Assume your HDFS cluster is made of 3 racks, each containing 3 DataNodes. Assume also the HDFS is configured to use a block size of 100 megabytes and that a client is connecting from outside the datacenter (therefore no DataNode is priviledged). 

1. The client uploads a file of 150 megabytes. Draw in the picture below a possible blocks configuration according to the default HDFS replica policy. How many replicas are there for each block? Where are these replicas stored?

1. Can you find a different policy that, using the same number of replicas, improves the expected availability of a block? Does your solution show any drawbacks?

1. Referring to the picture below, assume a block is stored in Node 3, as well as in Node 4 and Node 5. If this block of data has to be processed by a task running on Node 6, which of the three replicas will be actually read by Node 6? 

<img src="https://polybox.ethz.ch/index.php/s/lRzwDdtmytzyDRR/download" width="500">

**Solution**

1. For each block independently, the HDFS's placement policy is to put one replica on a random node in a random rack, another on one node in a different rack, and the last on a different node in the same rack chosen for the second replica. A possibile configuration is shown in the picture (but there are many more valid solutions).

1. One could decide to store the 3 replicas in 3 different racks, increasing the expected availability. However this would also slow down the writing process that would involve two inter-rack communications instead of one. Usually, the probability of failure of an entire rack is much smaller than the probability of failure of a node and therefore it is a good tradeoff to have 2/3 of the replicas in one rack.

1. Either the one stored in Node 4 or Node 5, assuming the intra-rack topology is such that the distance from these nodes to Node 6 is the same. In general, the reading priority is only based on the distance, not on which node was first selected in the writing process.

<img src="https://polybox.ethz.ch/index.php/s/7GSTXm0caYreggq/download" width="500">

### 2.2 &ndash; File read and write data flow.
To get an idea of how data flows between the client interacting with HDFS, consider a diagram below which shows main components of HDFS. 

<img src="https://polybox.ethz.ch/index.php/s/R7hg8x7YEyTFPvD/download" width="600">

1. Draw the main sequence of events when a client copies a file to HDFS.
2. Draw the main sequence of events when a client reads a file from HDFS.
3. Why do you think a client writes data directly to datanodes instead of sending it through the namenode?

**Solution**

1 - Steps 2-5 are applied for each block of the file. <br>
   1. HDFS client asks the Namenode to create the file.
   2. HDFS client asks the Namenode for a DataNode to host replica of the i-th block of the file. <br>
   3. NameNode replies with a list of DataNodes and their locations for i-th block. <br>
   4. The client writes i-th block to DataNodes in pipeline fashion. <br>
   5. DataNodes in the write pipeline acknowledge the writing of a block. Once all of them replied, the first contacted DataNode replies with acknowledgement to the client. <br>
   6. The client sends to the NameNode a request to close the file and release the lock. <br>
   7. The DataNodes check with the NameNode for minimal replication. <br>
   8. The NameNode sends ack to the client on finishing writing the file. <br>

<img src="https://polybox.ethz.ch/index.php/s/CvO26FssBV8eQ2M/download" width="500">

2 -  
   1. HDFS client request a file <br>
   2. NameNode replies with a list of blocks and the locations of each replica. <br>
   3. The client reads each block from the closest datanode.

<img src="https://polybox.ethz.ch/index.php/s/zxoqGqIIpvAg3Qv/download" width="500">

3 - If the namenode was responsible for copying all files to datanodes, then it would become a bottleneck.

### 2.3 &ndash; Network topology.

HDFS estimates the network bandwidth between two nodes by their distance. The distance from a node to its parent node is assumed to be one. A distance between two nodes can be calculated by summing up their distances to their closest common ancestor. A shorter distance between two nodes means that the greater bandwidth they can utilize to transfer data. Consider a diagram of a possible Hadoop cluster over two datacenters below. 

<img src="https://polybox.ethz.ch/index.php/s/Mk2kI7dkKZNrxul/download" width="700">

Calculate following distances using the distance rule explained above:

1. Node 0 and Node 1
2. Node 0 and Node 2
3. Node 1 and Node 4
4. Node 4 and Node 5
5. Node 2 and Node 3
6. Two processes of Node 1


**Solution**

1. 2
2. 4 
3. 6
4. 4
5. 2
6. 0

## 3. Storage models
### 3.1 &ndash; List two differences between Object Storage and Block Storage.

**Solution**

1. Block Storage implements file storage API, whereas Object Storage provides only key-value interface.

2. Pure Object Storage has a limit on object size, since the object cannot be partitioned across machines. Block Storage does not have this limitation and can split objects into blocks. Therefore, Block Storage can store PB files, whereas Object Storage is limited by the storage capacity of a single node. On the other hand, object storage can store more files than Block Storage.  

### 3.2 &ndash; Compare Object Storage and Block Storage. For each of the following use cases, say which technology better fits the requirements.

1. Store Netflix movie files in such a way they are accessible from many client applications at the same time [ *Object storage | Block Storage* ]

1. Store experimental and simulation data from CERN [ *Object storage | Block Storage* ]

1. Store the auto-backups of iPhone/Android devices [ *Object storage | Block Storage* ]


**Solution**

1. **Object Storage**. The movies are not excessively large to require Block Storage, while they can indefinitely in number, also a simple key-value model is enough without requiring a file-system hierarchy.

1. **Block Storage**. Because it can handle large files and store more data than ordinary object storage.

1. **Object Storage**. Backups are usually written once and rarely read. When data is read, partial access to each file is not essential. The client devices do not need to know the block composition of the object being stored. In fact, Apple [publicly confirmed](http://readwrite.com/2014/08/26/apple-icloud-amazon-web-services-hosting/) that backups data for iOS is stored on Amazon S3 and Microsoft Azure.

## 4. Working with Docker-Hadoop

Build and run the Hadoop docker image by `docker-compose up -d` in the `exercise03` directory. If completed successfully, you should be able to browse [`http://localhost:9870/`](http://localhost:9870/) and visualize the web interface of the daemon which should look similar to the following image. In the `Datanodes` tab you should see a single operating datanode.

<img src="https://polybox.ethz.ch/index.php/s/LpWcGWZeU5mipBK/download" width="800">


### Connecting to containers  

Each Hadoop cluster is set up in one of the three supported modes:

- Local (Standalone) Mode
- Pseudo-Distributed Mode
- Fully-Distributed Mode

By default Hadoop runs in Local Mode but we will run it in the *Pseudo-Distributed Mode*. This will allow you to run Hadoop on a single-node (your computer) simulating a distributed file system, with datanode and namenode running in separate containers. For this excercise you will only need to connect to `namenode` and `datanode` containers. To connect to namenode container can use the Docker dashboard interface by navigating to `docker-hadoop` app, and selecting `CLI` option from the `namenode` container (see image below).

<img src="https://polybox.ethz.ch/index.php/s/Hdlyhagx3JWbLBy/download" width="700">

Alternatively, you can run `docker exec -it namenode /bin/bash` in a terminal. To connect to a datanode, you can similarly find it in the dashboard or run `docker exec -it namenode /bin/bash` in the terminal. Both approaches will give you shell access on the corresponding container. 

### 4.1 &ndash; Upload a file into HDFS

Connect to the namenode by `docker exec -it namenode /bin/bash`.

Pick an image file in your computer (or you can also download a random one) and try to upload it to HDFS. You may need to create an empty directory before uploading. (Check [here](https://hadoop.apache.org/docs/current/hadoop-project-dist/hadoop-common/FileSystemShell.html) for help.)

1. Which command do you use to upload from the local file system to HDFS?

1. Which information can you find if you use `Utilities -> Browse the file system` in the daemon web interface?

**Solution**

1. You can follow the steps below:

From your local machine:
   ```bash
   # Transfer the file to the namenode container.
   docker cp path/to/pic.png namenode:pic.png
   ```

From inside the namenode container:

   ```bash
   # Add the file to HDFS from the namenode container.
   hadoop fs -mkdir /myfolder
   hadoop fs -put pic.png /myfolder
   ```

2. 
|Permission |	Owner |	Group |	Size |	Last Modified |	Replication |	Block Size	| Name|
|-----------|---------|-------|------|----------------|-------------|---------------|-----|
|-rw-r--r--	|anconam|	supergroup|	3.61 MB|	10/6/2016, 11:25:27 AM	|1|	128 MB	|pic.png|

### 4.2 &ndash; Local File System

1. ```bash
   docker cp docker-compose.yml namenode:docker-compose.yml 
   ```
Then, use HDFS commands to create a directory, copy the `docker-compose.yml` file from your local file system to HDFS. Use `cat` to check if the file is the same on the local and distributed systems. 

   *Hint:* you may use the following HDFS commands `-mkdir` for directory, `-copyFromLocal` for uploading the file, and `-cat` for printing them. You may have to first use `docker cp` to copy to file into the namenode container.

2. Try to locate the file on a datanode. To connect to a datanode by running:

   ```bash
   docker exec -it datanode /bin/bash
   ```

   This will give you shell access to the data node machine. cd into `/hadoop/dfs/data/current/` directory and follow the directories until there are only files. Can you check if the file contents are the same as the one you uploaded? Use `ls -l` to check the size of the file size on the local 

3. Now try to upload a file to HDFS that is ~150MB. On Unix-based system you can also generate such a file filled with zero using:

   ```bash
   dd if=/dev/zero of=zeros.dat bs=1M count=150
   ```

   How many blocks the file is split into?

**Solution**

1. You can use similar commands in HDFS to linux commands:

   ```bash
   hadoop fs -mkdir /myfolder
   hadoop fs -copyFromLocal docker-compose.yml /myfolder/
   hadoop fs -cat /myfolder/docker-compose.yml
   ```

   Uppon inspection, the local and distributed files are the same. 

2. For me the full path of the file is `/hadoop/dfs/data/current/BP-1800048097-172.18.0.6-1664731157189/current/finalized/subdir0/subdir0/blk_1073741825`. If we check if the content of the file is the same as the the text file that was uploaded. 

   The output of `ls -l` on the local file: 

   ```bash
   ls -l docker-compose.yml
   -rw-r--r-- 1 root root 2046 Sep 28 13:00 docker-compose.yml
   ```

   The output of `ls -l` on the data node:  
   ```bash
   ls -l blk_1073741825
   -rw-r--r-- 1 root root 2046 Oct  2 18:02 blk_1073741825
   ```

3. Two, one of about 128MB and the other about 22MB. Similar to before, we can use `hadoop fs -copyFromLocal zeros.dat /myfolder/` to upload the file. 

### 4.3 Demystifying FsImage & Edits, & Checkpoint

When the NameNode starts up, or a checkpoint is triggered by a configurable threshold:

- It reads the FsImage and EditLog from disk.
- It applies all the transactions from the EditLog to the in-memory representation of the FsImage.
- It flushes out this new version into a new FsImage on disk.
- It truncates the old EditLog because its transactions have been applied to the persistent FsImage.

A checkpoint can be triggered:

> at a given time interval (dfs.namenode.checkpoint.period) expressed in seconds,
> or after a given number of filesystem transactions have accumulated (dfs.namenode.checkpoint.txns).

If both of these properties are set, the first threshold to be reached triggers a checkpoint.

1. Query the configuration file

   - `hdfs getconf -confKey dfs.namenode.checkpoint.period`
   - `hdfs getconf -confKey dfs.namenode.checkpoint.txns`
   - The fsimage & edit logs location `hdfs getconf -confKey dfs.namenode.name.dir`, I get something like `file:///hadoop/dfs/name`
   - Find the fsimage and edit logs in the `current` directory. They must be named like `fsimage_0000000000000000000` & `edits_inprogress_0000000000000000001` 
   - Output edits `hdfs oev -p xml -i /hadoop/dfs/name/current/edits_inprogress_0000000000000000001 -o edits.xml `
   - Output fsimage `hdfs oiv -p XML -i /hadoop/dfs/name/current/fsimage_0000000000000000000 -o fsimage.xml`

2. Can you make sense of the outputs?

**Solution** 

1. Query the configuration file

   I get `3600s` for period and `100000` for transactions  

   Alternatively in the config file

   ```XML
   <property>
     <name>dfs.namenode.checkpoint.period</name>
     <value>3600s</value>
   </property>

   <property>
     <name>dfs.namenode.checkpoint.txns</name>
     <value>1000000</value>
   </property>
   ```

2. Can you make sense of the outputs? 

   The first 10 lines for fsImage that I get
   ```XML
   <?xml version="1.0"?>
   <fsimage><version><layoutVersion>-65</layoutVersion><onDiskVersion>1</onDiskVersion><oivRevision>b3cbbb467e22ea829b3808f4b7b01d07e0bf3842</oivRevision></version>
   <NameSection><namespaceId>602383739</namespaceId><genstampV1>1000</genstampV1><genstampV2>1000</genstampV2><genstampV1Limit>0</genstampV1Limit><lastAllocatedBlockId>1073741824</lastAllocatedBlockId><txid>0</txid></NameSection>
   <ErasureCodingSection>
   <erasureCodingPolicy>
   <policyId>1</policyId><policyName>RS-6-3-1024k</policyName><cellSize>1048576</cellSize><policyState>DISABLED</policyState><ecSchema>
   <codecName>rs</codecName><dataUnits>6</dataUnits><parityUnits>3</parityUnits></ecSchema>
   </erasureCodingPolicy>
   ```
   The first 20 lines I get for edits logs  
   ```XML
   <?xml version="1.0" encoding="UTF-8" standalone="yes"?>
   <EDITS>
     <EDITS_VERSION>-65</EDITS_VERSION>
     <RECORD>
       <OPCODE>OP_START_LOG_SEGMENT</OPCODE>
       <DATA>
         <TXID>1</TXID>
       </DATA>
     </RECORD>
     <RECORD>
       <OPCODE>OP_MKDIR</OPCODE>
       <DATA>
         <TXID>2</TXID>
         <LENGTH>0</LENGTH>
         <INODEID>16386</INODEID>
         <PATH>/myfolder</PATH>
         <TIMESTAMP>1633549304813</TIMESTAMP>
         <PERMISSION_STATUS>
           <USERNAME>root</USERNAME>
           <GROUPNAME>supergroup</GROUPNAME>
           <MODE>493</MODE>
         </PERMISSION_STATUS>
       </DATA>
     </RECORD>
   ```

### 4.4 Changing Block Size (optional)

As explained in the tutorials, to change HDFS configurations you edit `etc/hadoop/core-site.xml` and `etc/hadoop/hdfs-site.xml`. In the docker app, you can modify the variables in the `hadoop.env`. For example, in the following line,

```bash 
# hadoop.env 
CORE_CONF_fs_defaultFS=hdfs://namenode:9000
```

`CORE_CONF` corresponds to `core-site.xml`. The second part `fs_defaultFS=hdfs://namenode:9000` will be transformed into:

```xml
<property>
    <name>fs.defaultFS</name><
    value>hdfs://namenode:9000
    </value>
</property>
```

For more details [see here](https://github.com/big-data-europe/docker-hadoop).

Try changing the default block size of HDFS to see its affect on read & write performance. You can change the block size by modifying the follwoing line in `hadoop.env`: `HDFS_CONF_dfs_block_size=1048576` The value `1048576` determines the block size in bytes, which in this case is `2^20 bytes` or 1 megabytes.

> **_NOTE:_** that for these configuration changes to take effect you must restart the docker app!

1. Create a file with size ~150MB and uploade the file to HDFS. Check number of blocks via the Web interface. 

2. For each of the following block sizes 1048576, 134217728, measure the time to transfer from local to HDFS, from HDFS to local, and removing the file. 

You can run the following commands 
```bash
time hadoop fs -copyFromLocal zeros.dat /myfolder/zeros.dat
time hadoop fs -get /myfolder/zeros.dat zeros.dat
time hadoop fs -rm /myfolder/zeros.dat
```
Can you make sense of the results?

> **_NOTE:_** make sure to remove the files before uploading them, so that no caching will distort the measurements

**Solution**

1. There are 150 blocks, 

2. Here are the outputs on my machine (these are representative values, there is some variability)

   Blok size  | upload | download | remove |
   -----------|--------|----------|--------|
   1048576    | 7.363s |  3.188s  | 2.596s |
   134217728  | 3.550s |  2.599s  | 2.577s |

   Creating 150 blocks as opposed to 2 blocks incurs many separate file system calls that have a larger delay, while a 128 MB block is transfered in much fewer calls. 
   While this affects upload and download, it won't affect the remove time, since remove operation typically only affects metadata in the filesystem. 